In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}

dataset_df = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv', dtype=dtypes)
print("Full train dataset shape is {}".format(dataset_df.shape))

In [ ]:
dataset_df.head(5)

In [3]:
labels = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')

In [4]:
labels['session'] = labels.session_id.apply(lambda x: int(x.split('_')[0]) )
labels['q'] = labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )

In [60]:
CATEGORICAL = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']
NUMERICAL = ['elapsed_time','level','page','room_coor_x', 'room_coor_y', 
        'screen_coor_x', 'screen_coor_y', 'hover_duration']

NUMS = ['page', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y',
        'hover_duration', 'elapsed_time_diff']
DIALOGS = ['that', 'this', 'it', 'you','find','found','Found','notebook','Wells','wells','help','need', 'Oh','Ooh','Jo', 'flag', 'can','and','is','the','to']

name_feature = ['basic', 'undefined', 'close', 'open', 'prev', 'next']
event_names = ['cutscene_click', 'person_click', 'navigate_click',
       'observation_click', 'notification_click', 'object_click',
       'object_hover', 'map_hover', 'map_click', 'checkpoint',
       'notebook_click']

sub_room_lists = {
'0-4': ['tunic.historicalsociety.entry',
     'tunic.historicalsociety.stacks',
     'tunic.historicalsociety.basement',
     'tunic.kohlcenter.halloffame',
     'tunic.historicalsociety.collection',
     'tunic.historicalsociety.closet',
     'tunic.capitol_0.hall'],
'5-12': ['tunic.historicalsociety.entry',
     'tunic.library.frontdesk',
     'tunic.historicalsociety.frontdesk',
     'tunic.historicalsociety.stacks',
     'tunic.historicalsociety.closet_dirty',
     'tunic.humanecology.frontdesk',
     'tunic.historicalsociety.basement',
     'tunic.kohlcenter.halloffame',
     'tunic.library.microfiche',
     'tunic.drycleaner.frontdesk',
     'tunic.historicalsociety.collection',
     'tunic.capitol_1.hall',
     'tunic.capitol_0.hall'],
'13-22': ['tunic.historicalsociety.entry',
     'tunic.wildlife.center',
     'tunic.historicalsociety.cage',
     'tunic.library.frontdesk',
     'tunic.historicalsociety.frontdesk',
     'tunic.historicalsociety.stacks',
     'tunic.historicalsociety.closet_dirty',
     'tunic.humanecology.frontdesk',
     'tunic.historicalsociety.basement',
     'tunic.kohlcenter.halloffame',
     'tunic.library.microfiche',
     'tunic.drycleaner.frontdesk',
     'tunic.historicalsociety.collection',
     'tunic.flaghouse.entry',
     'tunic.historicalsociety.collection_flag',
     'tunic.capitol_1.hall',
     'tunic.capitol_2.hall'],
    }


sub_text_lists = {
'0-4': ['tunic.historicalsociety.entry.groupconvo',
     'tunic.historicalsociety.collection.cs',
     'tunic.historicalsociety.collection.gramps.found',
     'tunic.historicalsociety.closet.gramps.intro_0_cs_0',
     'tunic.historicalsociety.closet.teddy.intro_0_cs_0',
     'tunic.historicalsociety.closet.intro',
     'tunic.historicalsociety.closet.retirement_letter.hub',
     'tunic.historicalsociety.collection.tunic.slip',
     'tunic.kohlcenter.halloffame.plaque.face.date',
     'tunic.kohlcenter.halloffame.togrampa',
     'tunic.historicalsociety.collection.gramps.lost',
     'tunic.historicalsociety.closet.notebook',
     'tunic.historicalsociety.basement.janitor',
     'tunic.historicalsociety.stacks.outtolunch',
     'tunic.historicalsociety.closet.photo',
     'tunic.historicalsociety.collection.tunic',
     'tunic.historicalsociety.closet.teddy.intro_0_cs_5',
     'tunic.historicalsociety.entry.wells.talktogramps',
     'tunic.historicalsociety.entry.boss.talktogramps',
     'tunic.historicalsociety.closet.doorblock',
     'tunic.historicalsociety.entry.block_tomap2',
     'tunic.historicalsociety.entry.block_tocollection',
     'tunic.historicalsociety.entry.block_tomap1',
     'tunic.historicalsociety.collection.gramps.look_0',
     'tunic.kohlcenter.halloffame.block_0',
     'tunic.capitol_0.hall.chap1_finale_c',
     'tunic.historicalsociety.entry.gramps.hub'],
'5-12': ['tunic.historicalsociety.frontdesk.archivist.newspaper',
     'tunic.historicalsociety.frontdesk.archivist.have_glass',
     'tunic.drycleaner.frontdesk.worker.hub',
     'tunic.historicalsociety.closet_dirty.gramps.news',
     'tunic.humanecology.frontdesk.worker.intro',
     'tunic.library.frontdesk.worker.hello',
     'tunic.library.frontdesk.worker.wells',
     'tunic.historicalsociety.frontdesk.archivist.hello',
     'tunic.historicalsociety.closet_dirty.trigger_scarf',
     'tunic.drycleaner.frontdesk.worker.done',
     'tunic.historicalsociety.closet_dirty.what_happened',
     'tunic.historicalsociety.stacks.journals.pic_2.bingo',
     'tunic.humanecology.frontdesk.worker.badger',
     'tunic.historicalsociety.closet_dirty.trigger_coffee',
     'tunic.drycleaner.frontdesk.logbook.page.bingo',
     'tunic.library.microfiche.reader.paper2.bingo',
     'tunic.historicalsociety.closet_dirty.gramps.helpclean',
     'tunic.historicalsociety.frontdesk.archivist.have_glass_recap',
     'tunic.historicalsociety.frontdesk.magnify',
     'tunic.humanecology.frontdesk.businesscards.card_bingo.bingo',
     'tunic.library.frontdesk.wellsbadge.hub',
     'tunic.capitol_1.hall.boss.haveyougotit',
     'tunic.historicalsociety.basement.janitor',
     'tunic.historicalsociety.closet_dirty.photo',
     'tunic.historicalsociety.stacks.outtolunch',
     'tunic.library.frontdesk.worker.wells_recap',
     'tunic.capitol_0.hall.boss.talktogramps',
     'tunic.historicalsociety.closet_dirty.gramps.archivist',
     'tunic.historicalsociety.closet_dirty.door_block_talk',
     'tunic.historicalsociety.frontdesk.archivist.need_glass_0',
     'tunic.historicalsociety.frontdesk.block_magnify',
     'tunic.historicalsociety.frontdesk.archivist.foundtheodora',
     'tunic.historicalsociety.closet_dirty.gramps.nothing',
     'tunic.historicalsociety.closet_dirty.door_block_clean',
     'tunic.library.frontdesk.worker.hello_short',
     'tunic.historicalsociety.stacks.block',
     'tunic.historicalsociety.frontdesk.archivist.need_glass_1',
     'tunic.historicalsociety.frontdesk.archivist.newspaper_recap',
     'tunic.drycleaner.frontdesk.worker.done2',
     'tunic.humanecology.frontdesk.block_0',
     'tunic.library.frontdesk.worker.preflag',
     'tunic.drycleaner.frontdesk.worker.takealook',
     'tunic.library.frontdesk.worker.droppedbadge',
     'tunic.library.microfiche.block_0',
     'tunic.library.frontdesk.block_badge',
     'tunic.library.frontdesk.block_badge_2',
     'tunic.capitol_1.hall.chap2_finale_c',
     'tunic.drycleaner.frontdesk.block_0',
     'tunic.humanecology.frontdesk.block_1',
     'tunic.drycleaner.frontdesk.block_1'],
'13-22': ['tunic.historicalsociety.cage.confrontation',
     'tunic.wildlife.center.crane_ranger.crane',
     'tunic.wildlife.center.wells.nodeer',
     'tunic.historicalsociety.frontdesk.archivist_glasses.confrontation',
     'tunic.historicalsociety.basement.seescratches',
     'tunic.flaghouse.entry.flag_girl.hello',
     'tunic.historicalsociety.basement.ch3start',
     'tunic.historicalsociety.entry.groupconvo_flag',
     'tunic.historicalsociety.collection_flag.gramps.flag',
     'tunic.historicalsociety.basement.savedteddy',
     'tunic.library.frontdesk.worker.nelson',
     'tunic.wildlife.center.expert.removed_cup',
     'tunic.library.frontdesk.worker.flag',
     'tunic.historicalsociety.entry.boss.flag',
     'tunic.flaghouse.entry.flag_girl.symbol',
     'tunic.wildlife.center.wells.animals',
     'tunic.historicalsociety.cage.glasses.afterteddy',
     'tunic.historicalsociety.cage.teddy.trapped',
     'tunic.historicalsociety.cage.unlockdoor',
     'tunic.historicalsociety.stacks.journals.pic_2.bingo',
     'tunic.historicalsociety.entry.wells.flag',
     'tunic.humanecology.frontdesk.worker.badger',
     'tunic.historicalsociety.stacks.journals_flag.pic_0.bingo',
     'tunic.historicalsociety.entry.directory.closeup.archivist',
     'tunic.capitol_2.hall.boss.haveyougotit',
     'tunic.wildlife.center.wells.nodeer_recap',
     'tunic.historicalsociety.cage.glasses.beforeteddy',
     'tunic.wildlife.center.expert.recap',
     'tunic.historicalsociety.stacks.journals_flag.pic_1.bingo',
     'tunic.historicalsociety.cage.lockeddoor',
     'tunic.historicalsociety.stacks.journals_flag.pic_2.bingo',
     'tunic.wildlife.center.remove_cup',
     'tunic.wildlife.center.tracks.hub.deer',
     'tunic.historicalsociety.frontdesk.key',
     'tunic.library.microfiche.reader_flag.paper2.bingo',
     'tunic.flaghouse.entry.colorbook',
     'tunic.wildlife.center.coffee',
     'tunic.historicalsociety.collection_flag.gramps.recap',
     'tunic.wildlife.center.wells.animals2',
     'tunic.flaghouse.entry.flag_girl.symbol_recap',
     'tunic.historicalsociety.closet_dirty.photo',
     'tunic.historicalsociety.stacks.outtolunch',
     'tunic.historicalsociety.frontdesk.archivist_glasses.confrontation_recap',
     'tunic.historicalsociety.entry.boss.flag_recap',
     'tunic.capitol_1.hall.boss.writeitup',
     'tunic.library.frontdesk.worker.nelson_recap',
     'tunic.historicalsociety.entry.wells.flag_recap',
     'tunic.drycleaner.frontdesk.worker.done2',
     'tunic.library.frontdesk.worker.flag_recap',
     'tunic.library.frontdesk.worker.preflag',
     'tunic.historicalsociety.basement.gramps.seeyalater',
     'tunic.flaghouse.entry.flag_girl.hello_recap',
     'tunic.historicalsociety.basement.gramps.whatdo',
     'tunic.library.frontdesk.block_nelson',
     'tunic.historicalsociety.cage.need_glasses',
     'tunic.capitol_2.hall.chap4_finale_c',
     'tunic.wildlife.center.fox.concern']
}

SUB_LEVELS = {'0-4': [1, 2, 3, 4],
              '5-12': [5, 6, 7, 8, 9, 10, 11, 12],
              '13-22': [13, 14, 15, 16, 17, 18, 19, 20, 21, 22]}
level_groups = ["0-4", "5-12", "13-22"]

In [6]:
labels.head(5)

,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1


In [7]:
def feature_engineer(dataset_df):
    dfs = []
    for c in CATEGORICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        dfs.append(tmp)
    for c in NUMERICAL:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    return dataset_df

In [8]:
dataset_df = feature_engineer(dataset_df)
print("Full prepared dataset shape is {}".format(dataset_df.shape))

Full prepared dataset shape is (70686, 22)


In [9]:
def split_dataset(dataset, test_ratio=0.20):
    USER_LIST = dataset.index.unique()
    split = int(len(USER_LIST) * (1 - 0.20))
    return dataset.loc[USER_LIST[:split]], dataset.loc[USER_LIST[split:]]

train_x, valid_x = split_dataset(dataset_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_x), len(valid_x)))

56547 examples in training, 14139 examples in testing.


In [10]:
train_x.head(10)

,level_group,event_name_nunique,name_nunique,fqid_nunique,room_fqid_nunique,text_fqid_nunique,elapsed_time,level,page,room_coor_x,...,screen_coor_y,hover_duration,elapsed_time_std,level_std,page_std,room_coor_x_std,room_coor_y_std,screen_coor_x_std,screen_coor_y_std,hover_duration_std
session_id,,,,,,,,,,,,,,,,,,,,,
20090312431273200,0-4,10,3,30,7,17,8.579356e+04,1.945455,-1.000000,7.701275,...,383.044861,2389.500000,49246.539458,1.230975,-1.000000,399.296038,129.292411,214.871000,104.082743,3227.370757
20090312431273200,13-22,10,3,49,12,35,1.040601e+06,17.402381,-1.000000,-130.347168,...,379.301025,899.925903,126666.129584,2.358652,-1.000000,622.061374,230.370874,240.280218,99.067861,1305.088265
20090312431273200,5-12,10,3,39,11,24,3.572052e+05,8.054054,-1.000000,14.306062,...,378.784912,969.333313,80175.676658,2.096919,-1.000000,357.227701,137.409476,203.268560,120.255453,1316.408315
20090312433251036,0-4,11,4,22,6,11,9.763342e+04,1.870504,0.000000,-84.045959,...,370.723083,1378.750000,67372.714092,1.232616,0.000000,445.980041,156.186242,252.554707,121.062929,2114.876406
20090312433251036,13-22,11,6,73,16,43,2.498852e+06,17.762529,5.100000,-30.762283,...,387.930084,720.384949,777382.529186,1.825923,0.863075,529.575656,234.279590,259.288856,133.345693,1990.705518
20090312433251036,5-12,11,4,45,11,22,5.549040e+05,8.840506,2.142857,50.284168,...,375.771088,824.096802,159319.587112,2.134412,1.069045,377.074056,160.557098,232.280908,132.195573,1836.236232
20090312455206810,0-4,9,3,22,6,12,2.023980e+05,1.604027,-1.000000,-209.830322,...,481.268951,3145.666748,121848.824074,1.223745,-1.000000,478.652392,161.438387,459.750352,177.665582,4575.148668
20090312455206810,13-22,11,4,47,12,30,9.586433e+05,17.706044,5.230769,-58.655510,...,521.446716,1055.319946,125153.779971,2.462695,0.908083,642.936291,243.042697,395.599984,181.509084,1403.679502
20090312455206810,5-12,11,4,41,11,19,5.073953e+05,8.342857,2.333333,80.422424,...,510.330261,783.038452,76156.386207,2.181517,1.032796,363.422530,180.862289,346.987576,228.421676,825.029526


In [11]:
xgbparam = {
    1 : {'max_depth' : 8, 'subsample' : 0.5, 'colsample_bytree' : 1, 'n_estimators' : 80, 'learning_rate' : 0.01},
    2 : {'max_depth' : 4, 'subsample' : 0.5, 'colsample_bytree' : 1, 'n_estimators' : 100, 'learning_rate' : 0.01},
    3 : {'max_depth' : 8, 'subsample' : 0.75, 'colsample_bytree' : 0.95, 'n_estimators' : 800, 'learning_rate' : 0.02},
    4 : {'max_depth' : 8, 'subsample' : 0.7, 'colsample_bytree' : 0.75, 'n_estimators' : 800, 'learning_rate' : 0.02},
    5 : {'max_depth' : 8, 'subsample' : 0.5, 'colsample_bytree' : 0.75, 'n_estimators' : 800, 'learning_rate' : 0.01},
    6 : {'max_depth' : 8, 'subsample' : 0.6, 'colsample_bytree' : 0.9, 'n_estimators' : 400, 'learning_rate' : 0.02},
    7 : {'max_depth' : 4, 'subsample' : 0.6, 'colsample_bytree' : 0.9, 'n_estimators' : 400, 'learning_rate' : 0.02},
    8 : {'max_depth' : 6, 'subsample' : 0.7, 'colsample_bytree' : 0.9, 'n_estimators' : 400, 'learning_rate' : 0.02},
    9 :  {'max_depth' : 6, 'subsample' : 0.6, 'colsample_bytree' : 0.9, 'n_estimators' : 200, 'learning_rate' : 0.02},
    10 : {'max_depth' : 5, 'subsample' : 0.6, 'colsample_bytree' : 0.9, 'n_estimators' : 200, 'learning_rate' : 0.02},
    11 : {'max_depth' : 8, 'subsample' : 0.9, 'colsample_bytree' : 0.9, 'n_estimators' : 200, 'learning_rate' : 0.02},
    12 : {'max_depth' : 5, 'subsample' : 0.6, 'colsample_bytree' : 0.9, 'n_estimators' : 400, 'learning_rate' : 0.02},
    13 : {'max_depth' : 8, 'subsample' : 0.9, 'colsample_bytree' : 0.9, 'n_estimators' : 200, 'learning_rate' : 0.04},
    14 : {'max_depth' : 5, 'subsample' : 0.6, 'colsample_bytree' : 0.9, 'n_estimators' : 600, 'learning_rate' : 0.02},
    15 : {'max_depth' : 5, 'subsample' : 0.6, 'colsample_bytree' : 0.9, 'n_estimators' : 600, 'learning_rate' : 0.02},
    16 : {'max_depth' : 5, 'subsample' : 0.75, 'colsample_bytree' : 0.9, 'n_estimators' : 200, 'learning_rate' : 0.02},
    17 : {'max_depth' : 5, 'subsample' : 0.9, 'colsample_bytree' : 0.9, 'n_estimators' : 200, 'learning_rate' : 0.04},
    18 : {'max_depth' : 5, 'subsample' : 0.6, 'colsample_bytree' : 0.9, 'n_estimators' : 200, 'learning_rate' : 0.02}
}

In [39]:
models = {}
evaluation = {}
le = LabelEncoder()
# xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
for q_no in range(1,19):
    if q_no<=3: grp = '0-4'
    elif q_no<=13: grp = '5-12'
    elif q_no<=22: grp = '13-22'
    print("### q_no", q_no, "grp", grp)
    
    train_df = train_x.loc[train_x.level_group == grp]
    train_users = train_df.index.values
    valid_df = valid_x.loc[valid_x.level_group == grp]
    valid_users = valid_df.index.values

    # Select the labels for the related q_no.
    train_labels = labels.loc[labels.q==q_no].set_index('session').loc[train_users]
    valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]
    
    train_labels = le.fit_transform(train_labels["correct"])
    valid_labels = le.fit_transform(valid_labels["correct"])
    
    train_df = train_df.drop(columns=['level_group'])
    valid_df = valid_df.drop(columns=['level_group'])
    
    depths = [5, 6, 8]
    samples = [0.6, 0.75, 0.9]
    ests = [200, 400, 600, 800]
    rates = [0.02, 0.03, 0.04]
    
    # progress = tqdm(total=108)
    # best_score = 0
    '''for d in depths:
        for s in samples:
            for e in ests:
                for r in rates:
                    progress.update(1)
                    # xgboostModel = xgb.XGBClassifier(max_depth=d, subsample=s, colsample_bytree=0.9, n_estimators=e , learning_rate=r)
                    xgboostModel.set_params(**testparam[i])
                    xgboostModel.fit(train_df, train_labels)

                    models.append(xgboostModel)
                    train_score = xgboostModel.score(train_df, train_labels)
                    valid_score = xgboostModel.score(valid_df, valid_labels)
                    if valid_score > best_score:
                        best_score = valid_score
                        print('train: ',train_score)
                        print('valid: ',valid_score)
                        print(f"current params == depth: {d} ,samples: {s} ,ests: {e} ,rates:{r}")'''
    xgboostModel = xgb.XGBClassifier()
    xgboostModel.set_params(**xgbparam[q_no])
    xgboostModel.fit(train_df, train_labels)

    models[q_no] = xgboostModel
    train_score = xgboostModel.score(train_df, train_labels)
    valid_score = xgboostModel.score(valid_df, valid_labels)
    print('train: ',train_score)
    print('valid: ',valid_score)
    evaluation[q_no] = valid_score
# print("\nAverage accuracy", sum(evaluation.values()))

### q_no 1 grp 0-4
train:  0.7486338797814208
valid:  0.7318056439635052
### q_no 2 grp 0-4
train:  0.9796275664491485
valid:  0.9755994058985784
### q_no 3 grp 0-4
train:  0.9815374820945408
valid:  0.9350732017823042
### q_no 4 grp 5-12
train:  0.9377685818876333
valid:  0.7963080840229153
### q_no 5 grp 5-12
train:  0.8522998567563266
valid:  0.6291109696583916
### q_no 6 grp 5-12
train:  0.8642368295400287
valid:  0.7939741141523445
### q_no 7 grp 5-12
train:  0.7513395936123932
valid:  0.7485677912157861
### q_no 8 grp 5-12
train:  0.7165897395087273
valid:  0.6371737746658179
### q_no 9 grp 5-12
train:  0.7665128123507878
valid:  0.7706344154466369
### q_no 10 grp 5-12
train:  0.6517587139901321
valid:  0.6121366433269679
### q_no 11 grp 5-12
train:  0.7684757812085522
valid:  0.6609378315298111
### q_no 12 grp 5-12
train:  0.8649265213008648
valid:  0.8703585826437513
### q_no 13 grp 5-12
train:  0.8274709533662263
valid:  0.7237428389560789
### q_no 14 grp 13-22
train:  0.76932

In [41]:
for name, value in evaluation.items():
  print(f"question {name}: accuracy {value:.4f}")

print("\nAverage accuracy", sum(evaluation.values())/18)

question 1: accuracy 0.7318
question 2: accuracy 0.9756
question 3: accuracy 0.9351
question 4: accuracy 0.7963
question 5: accuracy 0.6291
question 6: accuracy 0.7940
question 7: accuracy 0.7486
question 8: accuracy 0.6372
question 9: accuracy 0.7706
question 10: accuracy 0.6121
question 11: accuracy 0.6609
question 12: accuracy 0.8704
question 13: accuracy 0.7237
question 14: accuracy 0.7371
question 15: accuracy 0.6191
question 16: accuracy 0.7490
question 17: accuracy 0.7049
question 18: accuracy 0.9516

Average accuracy 0.7581747884103073


In [58]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
thresholds = {}
th_evaluation = {}
for q_no in range(1,19):
    if q_no<=3: grp = '0-4'
    elif q_no<=13: grp = '5-12'
    elif q_no<=22: grp = '13-22'
    print("### q_no", q_no, "grp", grp)
    
    valid_df = valid_x.loc[valid_x.level_group == grp]
    valid_users = valid_df.index.values
    valid_df = valid_df.drop(columns=['level_group'])

    # Select the labels for the related q_no.
    valid_labels = labels.loc[labels.q==q_no].set_index('session').loc[valid_users]
    valid_labels = le.fit_transform(valid_labels["correct"])
    
    pred = models[q_no].predict_proba(valid_df)
    pred = pred[:, -1]
    
    best_score = 0
    best_threshold = 0
    for threshold in np.arange(0.2,0.8,0.01):
        valid_pred = (pred > threshold).astype(int)
        score = accuracy_score(valid_labels, valid_pred)
        if score > best_score: 
            best_score = score
            best_threshold = threshold
            th_evaluation[q_no] = accuracy_score(valid_labels, valid_pred)
    thresholds[q_no] = best_threshold
    print(best_threshold)

### q_no 1 grp 0-4
0.49000000000000027
### q_no 2 grp 0-4
0.2
### q_no 3 grp 0-4
0.5300000000000002
### q_no 4 grp 5-12
0.45000000000000023
### q_no 5 grp 5-12
0.5000000000000002
### q_no 6 grp 5-12
0.5100000000000002
### q_no 7 grp 5-12
0.5200000000000002
### q_no 8 grp 5-12
0.4300000000000002
### q_no 9 grp 5-12
0.5000000000000002
### q_no 10 grp 5-12
0.5000000000000002
### q_no 11 grp 5-12
0.5000000000000002
### q_no 12 grp 5-12
0.45000000000000023
### q_no 13 grp 5-12
0.5600000000000003
### q_no 14 grp 13-22
0.4100000000000002
### q_no 15 grp 13-22
0.5000000000000002
### q_no 16 grp 13-22
0.2
### q_no 17 grp 13-22
0.5000000000000002
### q_no 18 grp 13-22
0.2


In [59]:
for name, value in th_evaluation.items():
  print(f"question {name}: accuracy {value:.4f}")

print("\nAverage accuracy", sum(th_evaluation.values())/18)

question 1: accuracy 0.7333
question 2: accuracy 0.9756
question 3: accuracy 0.9355
question 4: accuracy 0.7972
question 5: accuracy 0.6291
question 6: accuracy 0.7948
question 7: accuracy 0.7490
question 8: accuracy 0.6384
question 9: accuracy 0.7706
question 10: accuracy 0.6121
question 11: accuracy 0.6609
question 12: accuracy 0.8704
question 13: accuracy 0.7248
question 14: accuracy 0.7371
question 15: accuracy 0.6191
question 16: accuracy 0.7494
question 17: accuracy 0.7049
question 18: accuracy 0.9516

Average accuracy 0.7585519956621167


In [14]:
import jo_wilder_310
env = jo_wilder_310.make_env()
iter_test = env.iter_test()

In [32]:
limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}

for (test, sample_submission) in iter_test:
    test = test.sort_values(by = 'index')
    test_df = feature_engineer(test)
    grp = test_df.level_group.values[0]
    a,b = limits[grp]
    for t in range(a,b):
        xgbm = models[t]
        test_ds = test_df.drop(columns=['level_group'])
        pred = xgbm.predict_proba(test_ds)[:, -1]
        mask = sample_submission.session_id.str.contains(f'q{t}')
        sample_submission.loc[mask,'correct'] = (pred > thresholds[t]).astype(int)
    
    env.predict(sample_submission)

In [33]:
! head submission.csv

session_id,correct
20090109393214576_q1,1
20090109393214576_q2,1
20090109393214576_q3,1
20090109393214576_q4,1
20090109393214576_q5,1
20090109393214576_q6,1
20090109393214576_q7,1
20090109393214576_q8,1
20090109393214576_q9,1
